###  Text Classification

**Loading the data set:**

The returned dataset is a scikit-learn “bunch”: a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, for instance the target_names holds the list of the requested category names:

In [126]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)



In [47]:
twenty_train.data[0].split("\n")

["From: lerxst@wam.umd.edu (where's my thing)",
 'Subject: WHAT car is this!?',
 'Nntp-Posting-Host: rac3.wam.umd.edu',
 'Organization: University of Maryland, College Park',
 'Lines: 15',
 '',
 ' I was wondering if anyone out there could enlighten me on this car I saw',
 'the other day. It was a 2-door sports car, looked to be from the late 60s/',
 'early 70s. It was called a Bricklin. The doors were really small. In addition,',
 'the front bumper was separate from the rest of the body. This is ',
 'all I know. If anyone can tellme a model name, engine specs, years',
 'of production, where this car is made, history, or whatever info you',
 'have on this funky looking car, please e-mail.',
 '',
 'Thanks,',
 '- IL',
 '   ---- brought to you by your neighborhood Lerxst ----',
 '',
 '',
 '',
 '',
 '']

In [45]:
twenty_train.target_names[0]

'alt.atheism'

You can check the target names (categories)

In [11]:
# You can check the target names (categories) and some data files by following commands.
twenty_train.target_names #prints all the categories



['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.
For speed and space efficiency reasons scikit-learn loads the target attribute as an array of integers that corresponds to the index of the category name in the target_names list. The category integer id of each sample is stored in the target attribute:

In [48]:
# from 11th label  to the end
twenty_train.target[10:]

array([ 8, 19,  4, ...,  3,  1,  8])

In [65]:
# from first label  to the eleventh
twenty_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [70]:
twenty_train.target[:20]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6,  0,  1,
        7, 12,  5])

In [69]:
 for t in twenty_train.target[:10]:
        print(twenty_train.target_names[t])

rec.autos
comp.sys.mac.hardware
comp.sys.mac.hardware
comp.graphics
sci.space
talk.politics.guns
sci.med
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
comp.sys.mac.hardware


In [7]:
twenty_train.target_names[:10]

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball']

In [16]:
#prints first   line of the first data file


print("\n".join(twenty_train.data[0].split("\n")[:3])) 

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


 Extracting features from text files.
Text files are actually series of words (ordered). In order to run machine learning algorithms we need to convert the text files into numerical feature vectors. We will be using bag of words model for our example. Briefly, we segment each text file into words (for English splitting by space), and count # of times each word occurs in each document and finally assign each word an integer id. Each unique word in our dictionary will correspond to a feature (descriptive feature).
Scikit-learn has a high level component which will create feature vectors for us ‘CountVectorizer’. More about it here.

#### Extracting features from text files

In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors.
##### Bags of words

The most intuitive way to do so is the bags of words representation:

* assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
* for each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary
The bags of words representation implies that n_features is the number of distinct words in the corpus: this number is typically larger than 100,000.

If n_samples == 10000, storing X as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM which is barely manageable on today’s computers.

Fortunately, **most values in X** will be zeros since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.
scipy.sparse matrices are data structures that do exactly this, and scikit-learn has built-in support for these structures.

#### Tokenizing text with scikit-learn

Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors:

In [3]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

**CountVectorizer** supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [71]:
count_vect.vocabulary_.get(u'algorithm')

27366

**sklearn.feature_extraction.text.CountVectorizer**

Convert a collection of text documents to a matrix of token counts
This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.

#### From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.
This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.
Both tf and tf–idf can be computed as follows:

** TF: ** Just counting the number of words in each document has 1 issue: it will give more weightage to longer documents than shorter documents. To avoid this, we can use frequency (TF - Term Frequencies) i.e. #count(word) / #Total words, in each document.
TF-IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) which occurs in all document. This is called as TF-IDF i.e Term Frequency times inverse document frequency.
We can achieve both using below line of code:

In [73]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

** Running ML algorithms **

There are various algorithms which can be used for text classification. We will start with the most simplest one ‘Naive Bayes (NB)’
You can easily build a NBclassifier in scikit using below 2 lines of code: (note - there are many variants of NB, but discussion about them is out of scope)

In [6]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)



Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:

In [129]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

Performance of NB Classifier: Now we will test the performance of the NB classifier on test set.

#### Naive Bayes Classifier

In [130]:
# Performance of NB Classifier
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)




0.7738980350504514

The accuracy is **77.39%**

#### Training Support Vector Machines 

In [19]:
# Training Support Vector Machines - SVM and calculating its performance

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(twenty_train.data, twenty_train.target)
predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)




0.82381837493361654

The accuracy is **82.38%**

#### Stochastic Gradient Descent Classifier

In [ ]:
import numpy as np
from sklearn import linear_model
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
Y = np.array([1, 1, 2, 2])
text_clf_sgd = linear_model.SGDClassifier()
text_clf_sgd=text_clf_sgd.clf.fit(X, Y)

SGDClassifier(loss="hinge", penalty="l2")

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42))])


text_clf_sgd = text_clf_sgd.fit(twenty_train.data, twenty_train.target)
predicted_sgd = text_clf_sgd.predict(twenty_test.data)
np.mean(predicted_sgd == twenty_test.target)



#### Parameter Tuning

**Grid Search**

Here, we are creating a list of parameters for which we would like to do performance tuning. 
All the parameters name start with the classifier name (remember the arbitrary name we gave). 
E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.



In [20]:
# Grid Search
# Here, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name (remember the arbitrary name we gave). 
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}



Next, we create an instance of the grid search by passing the classifier, parameters 
and n_jobs=-1 which tells to use multiple cores from user machine.

**Naive Bayes**

In [30]:
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

nbgs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
nbgs_clf = nbgs_clf.fit(twenty_train.data, twenty_train.target)



In [31]:
predicted = nbgs_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.83443972384492826

In [34]:

# To see the best mean score and the params, run the following code

nbgs_clf.best_score_
nbgs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

The accuracy  of the Naive Bayes classifier improves to **83.44%** after parameter tuning

**Support Vector Machines**

we get improved accuracy ~89.79% for SVM classifier with below code. Note: You can further optimize the SVM classifier by tuning other parameters. This is left up to you to explore more.

In [37]:
from sklearn.model_selection import GridSearchCV

parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf-svm__alpha': (1e-2, 1e-3),
 }
 
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [38]:
predicted = gs_clf_svm.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.83311205523101439

The accuracy  of the SVM  classifier improves to **83.31%** after parameter tuning

#### Useful tips and a touch of NLTK.
**Removing stop words: (the, then etc)** from the data. You should do this only when stop words are not useful for the underlying problem. In most of the text classification problems, this is indeed not useful. Let’s see if removing stop words increases the accuracy. Update the code for creating object of CountVectorizer as follows:

In [42]:
# NLTK
# Removing stop words
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)



0.81691449814126393

Removing stop words improves the accuracy to **81.69%**. We can include the stopwords in the  parameter tuning process for both Naive Bayes and SVM classifiers. 

* **FitPrior=False:** When set to false for MultinomialNB, a uniform prior will be used. 
* **Stemming:** From Wikipedia, stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form. E.g. A stemming algorithm reduces the words “fishing”, “fished”, and “fisher” to the root word, “fish”.
 NLTK comes with various stemmers (details on how stemmers work are out of scope for this article) which can help reducing the words to their root form. 

In [44]:
import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
 ])
text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)
predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)
np.mean(predicted_mnb_stemmed == twenty_test.target)

0.81678173127987252

The accuracy  of the Naive Bayes  classifier does not improve much to **81.67%**.

### Working With Text Data

To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

In order to get faster execution times for this first example we will work on a partial dataset with only 4 categories out of the 20 available in the dataset:

In [40]:
categories = ['alt.atheism', 'soc.religion.christian',
               'comp.graphics', 'sci.med']

In [41]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',categories=categories,
      shuffle=True, random_state=42)

twenty_test = fetch_20newsgroups(subset='test',categories=categories,
      shuffle=True, random_state=42)

#### Size of Files 

In [42]:
def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

data_train_size_mb = size_mb(twenty_train)
data_test_size_mb = size_mb(twenty_test)
print(data_train_size_mb)
print(data_test_size_mb)

4.7e-05
4.7e-05


The returned dataset is a scikit-learn “bunch”: a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, for instance the target_names holds the list of the requested category names:

In [4]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [5]:
print(len(twenty_train.data))

print(len(twenty_train.filenames))

2257
2257


Let’s print the first lines of the first loaded file:

In [220]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

print(twenty_train.target_names[twenty_train.target[0]])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
comp.graphics


Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.
For speed and space efficiency reasons scikit-learn loads the target attribute as an array of integers that corresponds to the index of the category name in the target_names list. The category integer id of each sample is stored in the target attribute:

In [6]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

It is possible to get back the category names as follows:

In [7]:
for t in twenty_train.target[:10]:
     print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


CountVectorizer supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [212]:
 count_vect.vocabulary_.get(u'algorithm')

27366

#### Tokenizing text with scikit-learn

Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors:

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

CountVectorizer supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [9]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [44]:
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

#### Training a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with a naïve Bayes classifier, which provides a nice baseline for this task. scikit-learn includes several variants of this classifier; the one most suitable for word counts is the multinomial variant:

In [45]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

In [46]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

#### Building a pipeline

In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [47]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

The names vect, tfidf and clf (classifier) are arbitrary. We shall see their use in the section on grid search, below. We can now train the model with a single command:

In [48]:
text_clf.fit(twenty_train.data, twenty_train.target) 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

#### Evaluation of the performance on the test set

Evaluating the predictive accuracy of the model is equally easy:

In [49]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.83488681757656458

I.e., we achieved 83.4% accuracy. Let’s see if we can do better with a linear support vector machine (SVM), which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by just plugging a different classifier object into our pipeline:

In [50]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),
 ])
text_clf.fit(twenty_train.data, twenty_train.target)  

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9127829560585885

scikit-learn further provides utilities for more detailed performance analysis of the results:

In [51]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
     target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



#### Tokenizing text with scikit-learn

Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors:

In [92]:
docs_new = ['God is love', 'OpenGL on the GPU is fast','Jesus Cchrist','laptop']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
predicted

array([15, 11, 15,  6])

In [93]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']


for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => sci.crypt
'Jesus Cchrist' => soc.religion.christian
'laptop' => misc.forsale


In [97]:
type(twenty_train.target_names)

list

In [114]:
map(twenty_train.target_names.__getitem__, (15,11))

In [111]:
#twenty_train.target_names[15,11]
from operator import itemgetter

itemgetter(15,11,15,6)(twenty_train.target_names)


('soc.religion.christian',
 'sci.crypt',
 'soc.religion.christian',
 'misc.forsale')

In [112]:
[ twenty_train.target_names[i] for i in predicted ]

['soc.religion.christian',
 'sci.crypt',
 'soc.religion.christian',
 'misc.forsale']

In [106]:
for w in predicted:
    print(twenty_train.target_names[w])  

soc.religion.christian
sci.crypt
soc.religion.christian
misc.forsale


In [119]:
import numpy as np

p=np.array(twenty_train.target_names)

p[predicted]

array(['soc.religion.christian', 'sci.crypt', 'soc.religion.christian',
       'misc.forsale'],
      dtype='<U24')

In [138]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])

text_clf.fit(twenty_train.data, twenty_train.target) 

import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.83488681757656458

In [142]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),
 ])

text_clf=text_clf.fit(twenty_train.data, twenty_train.target)  

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9127829560585885

#### Performance Analysis

scikit-learn further provides utilities for more detailed performance analysis of the results:

In [143]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
     target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [145]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]])

#### Classification of text documents using sparse features
This is an example showing how scikit-learn can be used to classify documents by topics using a bag-of-words approach. This example uses a scipy.sparse matrix to store the features and demonstrates various classifiers that can efficiently handle sparse matrices.
The dataset used in this example is the 20 newsgroups dataset. It will be automatically downloaded, then cached.
The bar plot indicates the accuracy, training time (normalized) and test time (normalized) of each classifier.

In [ ]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# parse commandline arguments
op = OptionParser()
op.add_option("--report",
              action="store_true", dest="print_report",
              help="Print a detailed classification report.")
op.add_option("--chi2_select",
              action="store", type="int", dest="select_chi2",
              help="Select some number of features using a chi-squared test")
op.add_option("--confusion_matrix",
              action="store_true", dest="print_cm",
              help="Print the confusion matrix.")
op.add_option("--top10",
              action="store_true", dest="print_top10",
              help="Print ten most discriminative terms per class"
                   " for every classifier.")
op.add_option("--all_categories",
              action="store_true", dest="all_categories",
              help="Whether to use all categories or not.")
op.add_option("--use_hashing",
              action="store_true",
              help="Use a hashing vectorizer.")
op.add_option("--n_features",
              action="store", type=int, default=2 ** 16,
              help="n_features when using the hashing vectorizer.")
op.add_option("--filtered",
              action="store_true",
              help="Remove newsgroup information that is easily overfit: "
                   "headers, signatures, and quoting.")


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

print(__doc__)
op.print_help()
print()


# #############################################################################
# Load some categories from the training set
if opts.all_categories:
    categories = None
else:
    categories = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space',
    ]

if opts.filtered:
    remove = ('headers', 'footers', 'quotes')
else:
    remove = ()

print("Loading 20 newsgroups dataset for categories:")
print(categories if categories else "all")

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)
print('data loaded')

# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names


def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

data_train_size_mb = size_mb(data_train.data)
data_test_size_mb = size_mb(data_test.data)

print("%d documents - %0.3fMB (training set)" % (
    len(data_train.data), data_train_size_mb))
print("%d documents - %0.3fMB (test set)" % (
    len(data_test.data), data_test_size_mb))
print("%d categories" % len(categories))
print()

# split a training set and a test set
y_train, y_test = data_train.target, data_test.target

print("Extracting features from the training data using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False,
                                   n_features=opts.n_features)
    X_train = vectorizer.transform(data_train.data)
else:
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    X_train = vectorizer.fit_transform(data_train.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_train.shape)
print()

print("Extracting features from the test data using the same vectorizer")
t0 = time()
X_test = vectorizer.transform(data_test.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_test.shape)
print()

# mapping from integer feature name to original token string
if opts.use_hashing:
    feature_names = None
else:
    feature_names = vectorizer.get_feature_names()

if opts.select_chi2:
    print("Extracting %d best features by a chi-squared test" %
          opts.select_chi2)
    t0 = time()
    ch2 = SelectKBest(chi2, k=opts.select_chi2)
    X_train = ch2.fit_transform(X_train, y_train)
    X_test = ch2.transform(X_test)
    if feature_names:
        # keep selected feature names
        feature_names = [feature_names[i] for i
                         in ch2.get_support(indices=True)]
    print("done in %fs" % (time() - t0))
    print()

if feature_names:
    feature_names = np.asarray(feature_names)


def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
        print()

    if opts.print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if opts.print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time


results = []
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (Perceptron(n_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False,
                                       tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                       penalty="elasticnet")))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,
                                                  tol=1e-3))),
  ('classification', LinearSVC(penalty="l2"))])))

# make some plots

indices = np.arange(len(results))

results = [[x[i] for x in results] for i in range(4)]

clf_names, score, training_time, test_time = results
training_time = np.array(training_time) / np.max(training_time)
test_time = np.array(test_time) / np.max(test_time)

plt.figure(figsize=(12, 8))
plt.title("Score")
plt.barh(indices, score, .2, label="score", color='navy')
plt.barh(indices + .3, training_time, .2, label="training time",
         color='c')
plt.barh(indices + .6, test_time, .2, label="test time", color='darkorange')
plt.yticks(())
plt.legend(loc='best')
plt.subplots_adjust(left=.25)
plt.subplots_adjust(top=.95)
plt.subplots_adjust(bottom=.05)

for i, c in zip(indices, clf_names):
    plt.text(-.3, i, c)

plt.show()


#### Stochastic Gradient Descent Classifier

In [37]:
'''
Created on May 5, 2017
'''

import re
import random
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

# Prepare data

def prepare_data(data):
    """
    data is expected to be a list of tuples of category and texts.
    Returns a tuple of a list of labels and a list of texts
    """
    random.shuffle(data)
    return zip(*data)

# Format training data

training_data = [
    ("good", "rain a lot the packs maybe damage."),
    ("good", "15107 Lane Pflugerville, TX customer called me and his phone number and my phone numbers were not masked. thank you customer has had a stroke and items were missing from his delivery the cleaning supplies for his wet vacuum steam cleaner.  he needs a call back from customer support "),
    ("gibber", "wh. screen"),
    ("gibber", "How will I know if I"),
    ("good", "I have problems scheduling blocks they are never any available.  Can I do full time?  Can I get scheduled more than one day a month?"),
    ("good", "Suggestion: easier way to sign in due alleviate the tediousness of periodically having to sign back in to the app to check for blocks."),
    ("good", "I am so glad to hear from you. "),
    ("good", "loading on today's itinerary takes ages!!!!!! time consuming when you have 150+ packages to deliver!!!!!"),
    ("good", "due to the new update that makes hours available at 10 pm. if you worked 8 hours that day you can't see next day hours due to 8 hour limit. please fix this"),
    ("good", "omg, PLEASE make it so we don't have to sign in every time we need to go into the app. At least make it good for a week. Thanks."),
    ("good", "Constantly being logged out of app, if we could have a continuous login so we could receive notifications if blocks are available that would be ideal."),
    ("good", "I am having problems  with the App. Every time I exit the App and reopen it asks for my login info."),
    ("good", "15 minute service time due to 33rd floor and  20l lbs of cargo"),
    ("good", "I have been sceduled 1 block in 3 weeks. I check for new block availability multiple times a day and have not seen 1 available in three weeks. is there any way to get more blocks."),
    ("good", "When will delivery jobs be available? Everytime I open this app, it says nothing is available. Have deliveries in Cincinnati started yet?"),
    ("good", "During delivery had to call customer support and after 10 minutes support person couldn't find my pick up location Kirkland /Bellevue and told me to hang up and call different support team.  Support person were unprofessional and rude, which is not acceptable."),
    ("good", "can you please remove the pick up from my phone"),
    ("good", "Dear friends: I'm very very happy it's a big oportunitt"),
    ("good", "THANK YOU so much for the block you assigned me for next week.   If you have an additional 5 blocks please go ahead and assign them to me for next week.  My availability is updated and current.  You guys are awesome!!!"),
    ("good", "after update every time I open app I have too log in! I used to be able to stay logged in unless I logged out, can you return stay logged in option."),
    ("good", "It looks like my app is not installed properly on my android phone, Note 5. I cannot access or do not see the tab to swipe to start delivering and the map or help button that should be visible for me to work today 5/6 at rpm"),
    ("gibber", "AF0000"),
    ("good", "awesome app, awesome hiring process, awesome delivery warehouse , awesome team and help in the field! lets deliver I would like more more more delivers , looking forward to the future ! I just bought a new delivery vehical !"),
    ("good", "I will like to ask why I can't get more delivery's only one in two weeks"),
    ("good", "device too slow software crashing all day"),
    ("good", "it doesn't work sometimes."),
    ("good", "can you please remove the old sprouts pick up from my phone"),
    ("good", "They ability to zoom in on text screens would be very helpful. Am example would be customer notes when viewing in certain lighting conditions can be difficult."),
    ("good", "I missed out on a delivery day when I clicked check in and waited for my turn to get an order only to find out that not only did my check in not register but the gps showed me down the street. I encountered this issue again when one of the warehouse employees placed an order for that location and the app wanted me to drive in a big circle to get back to where I was standing."),
    ("good", "i am a little concerned that i didn't receive any blocks of time for this coming week, even though i had a perfect delivery score from this past pay period. Did the Cincinnati market over hire drivers where there are many people being shut completely out of any delivery blocks for an entire week? i really enjoy this type of work and the app makes it quite convenient."),
    ("good", "I've arrived at the pick up restaurant but the staff did not have the barecode for me to scan, however I pick up the package and deliver but my is still not let me move on"),
    ("good", "might want to check my assigned hours for next week.  5am to 1pm??"),
    ("good", "hi team--just want to give some positive feedback.  I have had nothing but positive feedback from customers. Great support when calling help line. Thank you for this opportunity and if there is ever a situation where you need drivers immediately I will drop what I'm doing and help. You guys are the best."),
    ("good", "Allow days or blocks throughout the day to be modified after General availability is set up for time off like doctors appointments."),
    ("gibber", "AL001234"),
    ("good", "Please, enlight me."),
    ("good", "it only shows my schedule starting in two weeks. when will we be able to start work"),
    ("good", "include more packages for one block, if the packages can be fitted into the car, so driver don't have to come back and pickup every two hours. 25% of the time is wasted coming back for pick up."),
    ("gibber", "BBB h"),
    ("gibber", "AG0003006033SDgCJ12344"),
    ("gibber", "How will I know if I"),
    ("good", "please bring back some sort of hours cap! or possibly stagger the hour drops from 1200 to 1203 so that people with slower internet/slower phone arent at a disadvantage!"),
    ("good", "when the hours released tonight all of the people who didn't have 40 hours could see them.    however the drivers that are capped at 40 were unable to see them due to a flawed system.  please fix the system so that we are not continually treated unfairly like all of the drivers that whined so much and got us in to this mess.  the cap system is unfair to people that want to work and it caused problems with a lack of drivers  to deliver today at the hub.  obviously this is not a good system and benefits no one."),
    ("good", "You have seriously messed up the whole scheduling process. Why can't I get any blocks at 10 even if I wait exactly until 10? Midnight was much better. So now that scheduling is a huge random pain in the ass, why would people want to keep doing this? I haven't been able to schedule work for three days now, it's quite frustrating when I don't get a chance to sign up, even when I'm diligent with timing."),
    ("good", "Seriously, that's all I'm going to get is one lousy day? Tell me again why you need drivers if all we get is one day. I'm not sure this is gonna work out for me. I waited forever to get my background check back and this is what I get? smh"),
    ("good", "doesn't save updated access codes"),
    ("good", "the scheduling of my route is nor done very accurately. it keeps me driving back and forth"),
    ("good", "can't understand how to pick up a block. my availability is wide open. when you guys send the alerts about blocks available I open it real quick and there is nothing there. I do it in a matter of seconds"),
    ("good", "My availability keeps disappearing from my calender.   I set my availability for three weeks in advance. The gray dots are visible  but disappear on Wednesday or Thursday.  This makes it impossible for me to see and choose available blocks for the upcoming week. How can I get it fix.   Mike"),
    ("good", "GPS blank screen"),
    ("gibber", "sea swq"),
    ("gibber", "hiw o"),
    ("gibber", "Dr a"),
    ("gibber", "quick to quick to u uhu wu just us"),
    ("gibber", "Awa what's"),
    ("gibber", "wxdfcs"),
    ("gibber", "7k9opu"),
    ("gibber", "o.m.day day"),
    ("gibber", "GGT part his h"),
    ("gibber", "aawfhg"),
    ("gibber", "seesaw 2s"),
    ("gibber", "wawaa"),
    ("gibber", "of ll"),
    ("gibber", "rewards"),
    ("gibber", "mmqqm5my"),
    ("gibber", ".in w"),
    ("gibber", "play r"),
    ("gibber", "was wwnw www www n"),
    ("gibber", "wqq2fwqq2fz22"),
    ("gibber", "not"),
    ("gibber", "I by yu I"),
    ("gibber", "Hi just wanted to let you know that it's bee"),
    ("gibber", "I erroneously v"),
    ("gibber", "I find it"),
    ("gibber", "bqyyx I a"),
    ("gibber", "are are"),
    ("gibber", "wawi waarnnnkwn"),
    ("gibber", "t Petey ueteu he"),
    ("gibber", "ews ri"),
    ("gibber", "bd xd"),
    ("gibber", "hatpa"),
    ("gibber", "se wests tasgt"),
    ("gibber", "wa vgcx azc Jo of"),
    ("gibber", "2w222"),
    ("gibber", "her u t b"),
    ("gibber", "ddddedc"),
    ("gibber", "just juju in hiking"),
    ("gibber", "wew2ww2wwwew2i2wkkk"),
    ("gibber", "meleeee"),
    ("gibber", "Aaq wqXD"),


]
training_labels, training_texts = prepare_data(training_data)


# Format test data

test_data = [

("gibber", "an quality"),
    ("good", "Can't check in.   Time was 4:06.  I didn't drive out here for no reason."),
    ("good", "can you do view all full address including postal code how it's in old app that helps do correctly delivery and not waist customer time"),
    ("good", "i am available again starting at 10am to 10pm. thanks"),
    ("gibber", "Hello, I encountered"),
    ("good", "I want to know how we are notified if there is a block I have been signed in and haven't been given a block yet"),
    ("gibber", "aawaaw"),
    ("gibber", "eeeeeeeeene"),
    ("good", "I am not getting enough shifts"),
    ("gibber", "hey e75k"),
    ("good", "my screen had went black or inverted"),
    ("good", "maps packed up again in sr20ls"),
    ("good", "how to clear my itinerary from old pickup address ?"),
    ("good", "keep signing me out."),
    ("good", "For alcohol delivery,  where does customer sign?"),
    ("gibber", "t Petey ueteu he"),
    ("good", "can't get blocks.  too many drivers ??"),
    ("good", "got a new phone how do i download to new phone")



]
test_labels, test_texts = prepare_data(test_data)


# Create feature vectors

"""
Convert a collection of text documents to a matrix of token counts.
See: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
"""
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(training_texts)
y = training_labels




# Train the classifier
#fix the accuracy measure after each run
clf = SGDClassifier(random_state=5000)
#clf = SGDClassifier()
clf.fit(X, y)


# Test performance

X_test = vectorizer.transform(test_texts)
y_test = test_labels

# Generates a list of labels corresponding to the samples
test_predictionsSGDC = clf.predict(X_test)

# Convert back to the usual format
annotated_test_data = list(zip(test_predictionsSGDC, test_texts))
print(annotated_test_data)

# evaluate predictions
y_test = np.array(test_labels)
print(metrics.classification_report(y_test, test_predictions))
print("Accuracy: %0.4f" % metrics.accuracy_score(y_test, test_predictions))



[('gibber', 'Hello, I encountered'), ('good', "can't get blocks.  too many drivers ??"), ('gibber', 't Petey ueteu he'), ('gibber', 'i am available again starting at 10am to 10pm. thanks'), ('good', 'keep signing me out.'), ('gibber', 'got a new phone how do i download to new phone'), ('good', "I want to know how we are notified if there is a block I have been signed in and haven't been given a block yet"), ('good', 'my screen had went black or inverted'), ('good', 'For alcohol delivery,  where does customer sign?'), ('gibber', 'how to clear my itinerary from old pickup address ?'), ('gibber', 'aawaaw'), ('good', "can you do view all full address including postal code how it's in old app that helps do correctly delivery and not waist customer time"), ('good', 'an quality'), ('good', "Can't check in.   Time was 4:06.  I didn't drive out here for no reason."), ('gibber', 'eeeeeeeeene'), ('gibber', 'I am not getting enough shifts'), ('gibber', 'hey e75k'), ('gibber', 'maps packed up again

/Users/nanaakwasiabayieboateng/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


#### Naive Bayes

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_texts)

tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB().fit(X_train_tfidf, y)

#Generates a list of labels corresponding to the samples
test_predictionNB = clf1.predict(X_test)

#Convert back to the usual format
annotated_test_data = list(zip(test_predictionNB, test_texts))
print(annotated_test_data)
# evaluate predictions
y_test = np.array(test_labels)
print(metrics.classification_report(y_test, test_predictions))
print("Accuracy: %0.4f" % metrics.accuracy_score(y_test, test_predictions))




[('gibber', 'Hello, I encountered'), ('good', "can't get blocks.  too many drivers ??"), ('gibber', 't Petey ueteu he'), ('good', 'i am available again starting at 10am to 10pm. thanks'), ('good', 'keep signing me out.'), ('good', 'got a new phone how do i download to new phone'), ('good', "I want to know how we are notified if there is a block I have been signed in and haven't been given a block yet"), ('good', 'my screen had went black or inverted'), ('good', 'For alcohol delivery,  where does customer sign?'), ('good', 'how to clear my itinerary from old pickup address ?'), ('gibber', 'aawaaw'), ('good', "can you do view all full address including postal code how it's in old app that helps do correctly delivery and not waist customer time"), ('good', 'an quality'), ('good', "Can't check in.   Time was 4:06.  I didn't drive out here for no reason."), ('gibber', 'eeeeeeeeene'), ('good', 'I am not getting enough shifts'), ('gibber', 'hey e75k'), ('good', 'maps packed up again in sr20ls

### Sentiment Analyzer (Simple classification) with textblob

#### Installing/Upgrading From the PyPI

conda install -c https://conda.anaconda.org/sloria textblob

python -m textblob.download_corpora

**With conda**

conda install -c https://conda.anaconda.org/sloria textblob

python -m textblob.download_corpora

In [3]:
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger


train = [
    ('I love this sandwich.', 'pos'),
    ('This is an amazing place!', 'pos'),
    ('I feel very good about these beers.', 'pos'),
    ('This is my best work.', 'pos'),
    ("What an awesome view", 'pos'),
    ('I do not like this restaurant', 'neg'),
    ('I am tired of this stuff.', 'neg'),
    ("I can't deal with this", 'neg'),
    ('He is my sworn enemy!', 'neg'),
    ('My boss is horrible.', 'neg')
]
test = [
    ('The beer was good.', 'pos'),
    ('I do not enjoy my job', 'neg'),
    ("I ain't feeling dandy today.", 'neg'),
    ("I feel amazing!", 'pos'),
    ('Gary is a friend of mine.', 'pos'),
    ("I can't believe I'm doing this.", 'neg')
]

We create a new classifier by passing training data into the constructor for a NaiveBayesClassifier.

In [4]:
cl = NaiveBayesClassifier(train)

We can now classify arbitrary text using the NaiveBayesClassifier.classify(text) method.

In [6]:
cl.classify("Their burgers are amazing")   

'pos'

In [7]:
cl.classify("I don't like their pizza.") 

'neg'

Another way to classify strings of text is to use TextBlob objects. You can pass classifiers into the constructor of a TextBlob.



In [8]:
from textblob import TextBlob
blob = TextBlob("The beer was amazing. "
                "But the hangover was horrible. My boss was not happy.",
                classifier=cl)

You can then call the classify() method on the blob.

In [14]:
print(blob.classify())

neg


You can also take advantage of TextBlob's sentence tokenization and classify each sentence indvidually.

In [10]:
for sentence in blob.sentences:
    print(sentence)
    print(sentence.classify())

The beer was amazing.
pos
But the hangover was horrible.
neg
My boss was not happy.
neg


Let's check the accuracy on the test set.

In [13]:
# Compute accuracy
print("Accuracy: {0}".format(cl.accuracy(test)))

Accuracy: 0.8333333333333334


We can also find the most informative features:

In [15]:
# Show 5 most informative features
cl.show_informative_features(5)

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0


#### Adding More Data from NLTK

We can improve our classifier by adding more training and test data. Here we'll add data from the movie review corpus which was downloaded with NLTK.

In [18]:
import random
from nltk.corpus import movie_reviews

reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
new_train, new_test = reviews[0:100], reviews[101:200]

Let's see what one of these documents looks like.

In [23]:
print(new_train[0])

(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch', 'the', 'movie', 'and', '"', 'sorta', '"', 'find', 'out', '.', '.', '.', 'critique', ':', 'a', 'mind', '-', 'fuck', 'movie', 'for', 'the', 'teen', 'generation', 'that', 'touches', 'on', 'a', 'very', 'cool', 'idea', ',', 'but', 'presents', 'it', 'in', 'a', 'very', 'bad', 'package', '.', 'which', 'is', 'what', 'makes', 'this', 'review', 'an', 'even', 'harder', 'one', 'to', 'write', ',', 'since', 'i', 'generally', 'applaud', 'films', 'which', 'attempt', 'to', 'break', 'the', 'mold', ',', 'mess', 'with', 'your', 'head', 'and', 'such', '(', 'lost', 'highway', '&', 'memento', ')', ',', 'but', 'there', 'are', 'good', 'and', 'b

Notice that unlike the data in Part 1, the text comes as a list of words instead of a single string. TextBlob is smart about this; it will treat both forms of data as expected.

We can now update our classifier with the new training data using the update(new_data) method, as well as test it using the larger test dataset.

In [26]:
cl.update(new_train)
accuracy = cl.accuracy(test + new_test)

In [25]:
# Grab some movie review data
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(reviews)
new_train, new_test = reviews[0:100], reviews[101:200]

# Update the classifier with the new training data
cl.update(new_train)

# Compute accuracy
accuracy = cl.accuracy(test + new_test)
print("Accuracy: {0}".format(accuracy))

# Show 5 most informative features
cl.show_informative_features(5)


Accuracy: 0.5142857142857142
Most Informative Features
        contains(subtle) = True              pos : neg    =     11.9 : 1.0
   contains(wonderfully) = True              pos : neg    =     11.9 : 1.0
    contains(mysterious) = True              pos : neg    =      9.7 : 1.0
         contains(share) = True              pos : neg    =      9.7 : 1.0
        contains(namely) = True              pos : neg    =      9.7 : 1.0


#### Language Detector (Custom Feature Extraction)
An important aspect that I haven't yet mentioned is how features are being extracted from the text.

For a given document and training set train, TextBlob's default behavior is to compute which words in train are present in document. For example, the sentence "It's just a flesh wound." might have features contains(flesh): True, contains(wound): True, and contains(knight): False.

Of course, this simple feature extractor may not be appropriate for all problems. Here we'll create a custom feature extractor for a language detector.

Here's the training and test data.

In [27]:
train = [
    ("amor", "spanish"),
    ("perro", "spanish"),
    ("playa", "spanish"),
    ("sal", "spanish"),
    ("oceano", "spanish"),
    ("love", "english"),
    ("dog", "english"),
    ("beach", "english"),
    ("salt", "english"),
    ("ocean", "english")
]
test = [
    ("ropa", "spanish"),
    ("comprar", "spanish"),
    ("camisa", "spanish"),
    ("agua", "spanish"),
    ("telefono", "spanish"),
    ("clothes", "english"),
    ("buy", "english"),
    ("shirt", "english"),
    ("water", "english"),
    ("telephone", "english")
]

A feature extractor is simply a function that takes an argument text (the text to extract features from) and returns a dictionary of features.

Let's create a very simple extractor that uses the last letter of a given word as its only feature.

In [34]:
def extractor(word):
    feats = {}
    last_letter = word[-1]
    feats["last_letter({0})".format(last_letter)] = True
    return feats

print(extractor("python"))  # {'last_letter(n)': True}

{'last_letter(n)': True}


We can pass this feature extractor as the second argument to the constructor of a NaiveBayesClassifier.

In [36]:
lang_detector = NaiveBayesClassifier(train, feature_extractor=extractor)

And again, compute accuracy and informative features.

In [33]:
lang_detector.accuracy(test)  # 0.7
lang_detector.show_informative_features(5)

Most Informative Features
      last_letter(ocean) = None           spanis : englis =      1.2 : 1.0
      last_letter(beach) = None           spanis : englis =      1.2 : 1.0
     last_letter(oceano) = None           englis : spanis =      1.2 : 1.0
       last_letter(love) = None           spanis : englis =      1.2 : 1.0
      last_letter(playa) = None           englis : spanis =      1.2 : 1.0


In [38]:
from textblob import TextBlob

text = '''
The titular threat of The Blob has always struck me as the ultimate movie
monster: an insatiably hungry, amoeba-like mass able to penetrate
virtually any safeguard, capable of--as a doomed doctor chillingly
describes it--"assimilating flesh on contact.
Snide comparisons to gelatin be damned, it's a concept with the most
devastating of potential consequences, not unlike the grey goo scenario
proposed by technological theorists fearful of
artificial intelligence run rampant.
'''

blob = TextBlob(text)
blob.tags          

[('The', 'DT'),
 ('titular', 'JJ'),
 ('threat', 'NN'),
 ('of', 'IN'),
 ('The', 'DT'),
 ('Blob', 'NNP'),
 ('has', 'VBZ'),
 ('always', 'RB'),
 ('struck', 'VBN'),
 ('me', 'PRP'),
 ('as', 'IN'),
 ('the', 'DT'),
 ('ultimate', 'JJ'),
 ('movie', 'NN'),
 ('monster', 'NN'),
 ('an', 'DT'),
 ('insatiably', 'RB'),
 ('hungry', 'JJ'),
 ('amoeba-like', 'JJ'),
 ('mass', 'NN'),
 ('able', 'JJ'),
 ('to', 'TO'),
 ('penetrate', 'VB'),
 ('virtually', 'RB'),
 ('any', 'DT'),
 ('safeguard', 'NN'),
 ('capable', 'JJ'),
 ('of', 'IN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('doomed', 'JJ'),
 ('doctor', 'NN'),
 ('chillingly', 'RB'),
 ('describes', 'VBZ'),
 ('it', 'PRP'),
 ('assimilating', 'VBG'),
 ('flesh', 'NN'),
 ('on', 'IN'),
 ('contact', 'NN'),
 ('Snide', 'JJ'),
 ('comparisons', 'NNS'),
 ('to', 'TO'),
 ('gelatin', 'VB'),
 ('be', 'VB'),
 ('damned', 'VBN'),
 ('it', 'PRP'),
 ("'s", 'VBZ'),
 ('a', 'DT'),
 ('concept', 'NN'),
 ('with', 'IN'),
 ('the', 'DT'),
 ('most', 'RBS'),
 ('devastating', 'JJ'),
 ('of', 'IN'),
 ('potenti

In [41]:


blob.noun_phrases  

WordList(['titular threat', 'blob', 'ultimate movie monster', 'amoeba-like mass', 'snide', 'potential consequences', 'grey goo scenario', 'technological theorists fearful', 'artificial intelligence run rampant'])

In [42]:

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)


0.06000000000000001
-0.34166666666666673


In [43]:
blob.translate(to="es")  # to spanish

TextBlob("La amenaza principal de The Blob siempre me ha parecido la mejor película
monstruo: una masa insaciablemente hambrienta, similar a una ameba capaz de penetrar
prácticamente cualquier salvaguardia, capaz de - como un doctor condenado escalofriante
lo describe - "asimilando carne en contacto.
Las malditas comparaciones con la gelatina pueden ser condenadas, es un concepto con la mayor cantidad de
devastador de posibles consecuencias, a diferencia del escenario gris goo
propuesto por teóricos tecnológicos temerosos de
la inteligencia artificial corre desenfrenada.")

In [44]:
blob.translate(to="fr")  #to french

TextBlob("La menace de The Blob m'a toujours frappé comme le film ultime
monstre: une masse insatiablement affamée et amibienne capable de pénétrer
pratiquement toute sauvegarde, capable de - comme un médecin condamné à froid
le décrit - "assimiler la chair au contact.
Les comparaisons de Snide à la gélatine soient damnées, c'est un concept avec le plus
dévastateur de conséquences potentielles, pas différent du scénario gris goo
proposé par les théoriciens de la technologie peur de
l'intelligence artificielle sévit.")

#### Work with libraries in Python with r2py library

In [3]:
from rpy2.robjects import r

ModuleNotFoundError: No module named 'rpy2'